The aim of this notebook is to visualize the effects of some of the transformations.

In [ ]:
import sys 
sys.path.append('..')

In [ ]:
import matplotlib.pyplot as plt

from monai.data import DataLoader, Dataset
from monai.utils import first
import monai.transforms as T

In [ ]:
base_transforms = [
    T.LoadImaged(
        keys=['img']
    ),
    T.EnsureChannelFirstd(
        keys=['img']
    ),
    T.Orientationd(
        keys=['img'], 
        axcodes='RAS'
    ),
    T.Spacingd(
        keys=['img'], 
        pixdim=(1, 1, 2.5),
        mode=('bilinear')
    ),
    T.ScaleIntensityRanged(
        keys=['img'], 
        a_min=-500, a_max=500,
        b_min=0.0, b_max=1.0, clip=True
    ),
    T.CropForegroundd(
        keys=['img'], 
        source_key='img'
    ),
    T.SpatialPadd(
        keys=['img'], 
        spatial_size=(96, 96, 96)
    ),
    T.EnsureTyped(
        keys=['img'], 
        track_meta=False
    )
]

transforms = T.Compose([
    *base_transforms,
    T.RandZoomd(
        keys=['img'], 
        min_zoom=0.8, 
        max_zoom=1.2, 
        mode=('trilinear'),
        align_corners=True,
        prob=1
    ),
    # T.RandCropByPosNegLabeld(
    #     keys=['img'],
    #     label_key='label',
    #     spatial_size=(96, 96, 96),
    #     pos=1,
    #     neg=1,
    #     num_samples=2,
    #     image_key='img',
    #     image_threshold=0
    # ),
    # T.RandGaussianSmoothd(
    #     keys=['img'],
    #     prob=0.1
    # ),
    T.RandScaleIntensityd(
        keys=['img'], 
        factors=0.1, 
        prob=1
    ),
    T.RandShiftIntensityd(
        keys=['img'], 
        offsets=0.1, 
        prob=1
    ),
    # T.RandGaussianNoised(
    #     keys=['img'],
    #     std=0.01,
    #     prob=0.15
    # ),
    # T.RandFlipd(
    #     keys=['img'], 
    #     spatial_axis=0,
    #     prob=0.25
    # ),
    # T.RandFlipd(
    #     keys=['img'], 
    #     spatial_axis=1,
    #     prob=0.25
    # ),
    # T.RandFlipd(
    #     keys=['img'], 
    #     spatial_axis=2,
    #     prob=0.25
    # ),
    # T.RandRotate90d(
    #     keys=['img'], 
    #     max_k=3,
    #     prob=0.25
    # )
])

In [ ]:
data = [
    {
        'img': '../data/ssl/FLARETs_0047_0000.nii.gz'
    },
    {
        'img': '../data/ssl/FLARETs_0048_0000.nii.gz'
    },
    {
        'img': '../data/ssl/FLARETs_0049_0000.nii.gz'
    }
]

In [ ]:
check_ds = Dataset(data=data, transform=transforms)
check_loader = DataLoader(check_ds, batch_size=1)

In [ ]:
check_data = first(check_loader)
img = check_data['img'][0][0]

In [ ]:
plt.imshow(check_data['img'][0][0][:, :, 50], cmap='gray', vmin=0, vmax=1)

In [ ]:
plt.imshow(check_data['img'][0][0][:, :, 0], cmap='gray', vmin=0, vmax=1)

In [ ]:
print(check_data['img1_coords'])
plt.imshow(check_data['img1'][0][0][:, :, 0], cmap='gray', vmin=0, vmax=1)

In [ ]:
print(check_data['img2_coords'])
plt.imshow(check_data['img2'][0][0][:, :, 7], cmap='gray', vmin=0, vmax=1)